# Mintpy Time Series
George Brencher

In [ ]:
# Clone MintPy, create environment, install MintPy. Environment creation takes 10-15 min
# git clone https://github.com/insarlab/MintPy.git
!mamba env create -f MintPy/docs/environment.yml
!conda activate mintpy
!python -m ipykernel install --user --name=mintpy

In [ ]:
# Import required packages
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
import mintpy
from mintpy.objects import ifgramStack
from mintpy.utils import plot as pp, utils as ut
from mintpy import view, plot_network
from mintpy.unwrap_error_phase_closure import plot_num_triplet_with_nonzero_integer_ambiguity

In [ ]:
# download example dataset, since topsStack is not working correctly
if not os.path.exists('/tmp/data'):
        os.makedirs('/tmp/data')

os.chdir('/tmp/data')

In [ ]:
# grabbing example interferogram stack
url = 'https://zenodo.org/record/3952953/files/FernandinaSenDT128.tar.xz'
tar_file = os.path.basename(url)
proj_name = tar_file.split('.')[0]
if not os.path.isfile(tar_file):
    !wget {url}
else:
    print('{} exists, skip re-download.'.format(tar_file))

# uncompress example dataset
if not os.path.isdir(proj_name):
    print('uncompressing the tar file ...')
    !tar -xJf {tar_file}

In [ ]:
# create directory for MintPy
os.chdir('/tmp')

if not os.path.exists('/tmp/mintpy'):
        os.makedirs('/tmp/mintpy')

os.chdir('/tmp/mintpy')

In [ ]:
# Function to write to MintPy config file
def write_config_file(out_file, CONFIG_TXT, mode='a'): 
    """Write configuration files for MintPy to process ISCE products"""
    if not os.path.isfile(out_file) or mode == 'w':
        with open(out_file, "w") as fid:
            fid.write(CONFIG_TXT)
        print('write configuration to file: {}'.format(out_file))
    else:
        with open(out_file, "a") as fid:
            fid.write("\n" + CONFIG_TXT)
        print('add the following to file: \n{}'.format(CONFIG_TXT))

In [ ]:
# Write config file for MintPy. Needs to be parameterized with paths and desired options.
# For a list of processing options, see: 
#https://nbviewer.org/github/insarlab/MintPy-tutorial/blob/main/smallbaselineApp.ipynb
CONFIG_TXT = '''# vim: set filetype=cfg:
##----------------------------- SentinelStack/ISCE ---------------------##
#ssaraopt = --platform=SENTINEL-1A,SENTINEL-1B -r 128 -f 589,590,591,592,593  -e 2017-07-01
#sentinelStack.boundingBox      = '-1 0.15 -91.6 -90.9'
#sentinelStack.subswath         = 1 2     # comment 
#sentinelStack.numConnections   = 5       # comment
#sentinelStack.azimuthLooks     = 5       # comment
#sentinelStack.rangeLooks       = 15      # comment
#sentinelStack.filtStrength     = 0.2     # comment
#sentinelStack.unwMethod        = snaphu  # comment
#sentinelStack.coregistration   = auto    # comment
#subset.y0:y1,x0:x1 = 1150:1600,1070:1670

##-------------------------------- MintPy -----------------------------##
########## 1. Load Data (--load to exit after this step)
## load_data.py -H to check more details and example inputs.
mintpy.load.processor        = isce
##---------for ISCE only:
mintpy.load.metaFile         = ../data/FernandinaSenDT128/reference/IW*.xml
mintpy.load.baselineDir      = ../data/FernandinaSenDT128/baselines
##---------interferogram datasets:
mintpy.load.unwFile          = ../data/FernandinaSenDT128/merged/interferograms/*/filt_*.unw
mintpy.load.corFile          = ../data/FernandinaSenDT128/merged/interferograms/*/filt_*.cor
mintpy.load.connCompFile     = ../data/FernandinaSenDT128/merged/interferograms/*/filt_*.unw.conncomp
##---------geometry datasets:
mintpy.load.demFile          = ../data/FernandinaSenDT128/merged/geom_reference/hgt.rdr
mintpy.load.lookupYFile      = ../data/FernandinaSenDT128/merged/geom_reference/lat.rdr
mintpy.load.lookupXFile      = ../data/FernandinaSenDT128/merged/geom_reference/lon.rdr
mintpy.load.incAngleFile     = ../data/FernandinaSenDT128/merged/geom_reference/los.rdr
mintpy.load.azAngleFile      = ../data/FernandinaSenDT128/merged/geom_reference/los.rdr
mintpy.load.shadowMaskFile   = ../data/FernandinaSenDT128/merged/geom_reference/shadowMask.rdr
mintpy.load.waterMaskFile    = None

mintpy.reference.lalo        = -0.30,-91.43
mintpy.topographicResidual.stepFuncDate  = 20170910,20180613  #eruption dates
mintpy.deramp                = linear
'''

config_file = os.path.join('/tmp/mintpy', "FernandinaSenDT128.txt")
write_config_file(config_file, CONFIG_TXT, mode='w')

In [ ]:
# View MintPy options
!smallbaselineApp.py --help

In [ ]:
# Load data into MintPy
!smallbaselineApp.py FernandinaSenDT128.txt --dostep load_data

In [ ]:
# Examine loaded inputs to MintPy
!ls -l inputs

In [ ]:
# Plot all data related to one interferometric pair using MintPy plotting utility
view.main('./inputs/ifgramStack.h5 20180526_20180619 --ncols 3 -c viridis'.split())

In [ ]:
# Examine radar geometry files
view.main('./inputs/geometryRadar.h5 -c viridis'.split())

In [ ]:
# Optional step prior to inversion. Remove interferograms based on coherence, temporal/perpendicular baselines, date, etc. 
!smallbaselineApp.py FernandinaSenDT128.txt --dostep modify_network

In [ ]:
# Examine interferogram network using MintPy tool
plot_network.main(['./inputs/ifgramStack.h5'])

In [ ]:
# Select reference point. Should be coherent and close to the aoi. 
!smallbaselineApp.py FernandinaSenDT128.txt --dostep reference_point

In [ ]:
# Without inversion, quickly assesses possible deformation and phase unwrapping error by stacking phase 
!smallbaselineApp.py FernandinaSenDT128.txt --dostep quick_overview

In [ ]:
# Plot results of phase stacking 
view.main('avgPhaseVelocity.h5 -c RdBu -v -10 10'.split())

In [ ]:
# Plot number of triplets with nonzero integer ambiguity, related to unwrapping error
plot_num_triplet_with_nonzero_integer_ambiguity('numTriNonzeroIntAmbiguity.h5', display=True, fig_size=[12, 4])

In [ ]:
# Do time series inversion. 
!smallbaselineApp.py FernandinaSenDT128.txt --dostep invert_network

In [ ]:
# Plot the raw phase time-series re-wrapped into (-5, 5) cm
view.main('timeseries.h5 --wrap --wrap-range -5 5 -u cm --notitle --notick --noaxis -c RdBu -v -5 5'.split())

In [ ]:
# Plot time series temporal coherence and mask coherence 
# Unclear to me why border is sawtooth. Will investigate further in future
view.main('temporalCoherence.h5 -c gray --notick --noaxis --noverbose'.split())
view.main('maskTempCoh.h5 -c gray --notick --noaxis --noverbose'.split())

In [ ]:
# Correct for tropospheric correction. Copernicus Climate Data Store (CDS) registration is needed to access ERA5 data. 
# To create account and set up api, follow steps at https://github.com/insarlab/pyaps#2-account-setup-for-era5
###### NOTE: atmopsheric correction yielding less realistic results for this time series, step is skipped
#!smallbaselineApp.py FernandinaSenDT128.txt --dostep correct_troposphere

In [ ]:
# View tropospheric corrected time series
#view.main('timeseries_ERA5.h5 --wrap --wrap-range -5 5 --notitle --notick --noaxis -c RdBu'.split())

In [ ]:
# Remove linear or quadratic ramps based on the phase of reliable pixels. Recommended for localized deformation signals
!smallbaselineApp.py FernandinaSenDT128.txt --dostep deramp

In [ ]:
# View deramped time series
view.main('timeseries_ERA5_ramp.h5 --wrap --wrap-range -5 5 --notitle --notick --noaxis -c RdBu'.split())

In [ ]:
# Correct phase residual caused by DEM error
!smallbaselineApp.py FernandinaSenDT128.txt --dostep correct_topography

In [ ]:
# View DEM error-corrected time series
view.main('timeseries_ERA5_ramp_demErr.h5 --wrap --wrap-range -5 5 --notitle --notick --noaxis -c RdBu'.split())

In [ ]:
# Calculate average velocity of time series 
!smallbaselineApp.py FernandinaSenDT128.txt --dostep velocity

In [ ]:
# View original and troposphere corrected velocities
view.main('velocity.h5 --notick --noaxis --noverbose -c RdBu -v -5 5'.split())
#view.main('velocityERA.h5 --notick --noaxis --noverbose -c RdBu -v -5 5'.split())

In [ ]:
# Geocode outputs
!smallbaselineApp.py FernandinaSenDT128.txt --dostep geocode

In [ ]:
# View geocoded outputs
!ls -l ./geo

In [ ]:
# Plot geocoded velocity 
view.main('./geo/geo_velocity.h5 velocity --dem ./geo/geo_geometryRadar.h5 --shade-exag 0.05 --figsize 12 12 --notitle --notick --noaxis -c RdBu -v -5 5'.split())